In [1]:
import sqlite3

con = sqlite3.connect("./DB/stockPrice.db")
cur  = con.cursor()

create_table_sql ="""
CREATE TABLE stockPrice (code TEXT,date TEXT, open REAL, high REAL, low REAL, close REAL,adjclose REAL,volume REAL,PRIMARY KEY(code, date))
"""

cur.execute(create_table_sql)

In [76]:
import pandas as pd
import yfinance


def getStockPriceInfo(code:str,period:str,interval:str) -> pd.core.frame.DataFrame:

    # DataFrameを作成します
    table_name = "stockPrice"
    df = yfinance.download(str(code) + ".T", period=period,interval=interval)

    #indexを最初の列に挿入
    df.reset_index(inplace= True)
    df = df.rename(columns={'index': 'date'})
    #codeを最初の列に挿入
    df.insert(0, 'code', str(code))

    #列名の変更
    df.columns = ['code', 'date', 'open', 'high', 'low', 'close', 'adjclose', 'Volume']

    #'date'列をdatetime->strへ型変換
    df['date'] = df['date'].astype(str)

    #'-'はバイト数を消費するので削除しておく。
    df['date'] = df['date'].str.replace('-', '')

    #adjclose列の小数点第2位以下を丸め込む
    df['adjclose'] = df['adjclose'].round(2)
    
    return df

df = getStockPriceInfo(code='1301',period='1mo',interval='1d')

[*********************100%%**********************]  1 of 1 completed


In [75]:
df

,code,date,open,high,low,close,adjclose,Volume
0,1301,20240304,3580.0,3580.0,3500.0,3510.0,3418.5,1021000
1,1301,20240305,3525.0,3550.0,3515.0,3530.0,3438.0,164500
2,1301,20240306,3575.0,3670.0,3560.0,3665.0,3569.4,247500
3,1301,20240307,3660.0,3690.0,3625.0,3630.0,3535.3,106900
4,1301,20240308,3635.0,3700.0,3625.0,3700.0,3603.5,112800
5,1301,20240311,3700.0,3700.0,3620.0,3655.0,3559.7,116700
6,1301,20240312,3660.0,3660.0,3600.0,3635.0,3540.2,79200
7,1301,20240313,3660.0,3675.0,3615.0,3630.0,3535.3,60800
8,1301,20240314,3650.0,3695.0,3635.0,3695.0,3598.7,49100
9,1301,20240315,3690.0,3725.0,3690.0,3705.0,3608.4,61500


In [70]:
#connection確立
con = sqlite3.connect("./DB/stockPrice.db")

#一括書込み
df.to_sql('stockPrice',con,if_exists='append',index=None)

#commit
con.commit()

con.close()

IntegrityError: UNIQUE constraint failed: stockPrice.code, stockPrice.date

In [71]:
con = sqlite3.connect("./DB/stockPrice.db")

sql = '''
 SELECT date FROM stockprice 
'''
res = con.execute(sql)
res.fetchall()

[('20240304',),
 ('20240305',),
 ('20240306',),
 ('20240307',),
 ('20240308',),
 ('20240311',),
 ('20240312',),
 ('20240313',),
 ('20240314',),
 ('20240315',),
 ('20240318',),
 ('20240319',),
 ('20240321',),
 ('20240322',),
 ('20240325',),
 ('20240326',),
 ('20240327',),
 ('20240328',),
 ('20240329',),
 ('20240401',),
 ('20240402',),
 ('2024-03-04',),
 ('2024-03-05',),
 ('2024-03-06',),
 ('2024-03-07',),
 ('2024-03-08',),
 ('2024-03-11',),
 ('2024-03-12',),
 ('2024-03-13',),
 ('2024-03-14',),
 ('2024-03-15',),
 ('2024-03-18',),
 ('2024-03-19',),
 ('2024-03-21',),
 ('2024-03-22',),
 ('2024-03-25',),
 ('2024-03-26',),
 ('2024-03-27',),
 ('2024-03-28',),
 ('2024-03-29',),
 ('2024-04-01',),
 ('2024-04-02',)]

In [35]:
import pandas as pd
import re
import pandas as pd
import urllib.request

#セキュリティ的にだめ！！（一時凌ぎ）
#https://qiita.com/shutokawabata0723/items/9733a7e640a175c23f39
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def getStockCodeDataFrame() -> pd.core.frame.DataFrame:

    URL = 'https://www.jpx.co.jp/markets/statistics-equities/misc/01.html'


    response = urllib.request.urlopen(URL).read().decode("utf-8")
    string_html = re.findall('<a href=\".+?\.xls\"',response)
    url_list = []
    for i in string_html:
        j = i.lstrip('<a href=\"')
        k = j.rstrip('\"')
        url_list.append('https://www.jpx.co.jp'+k)
    url = url_list[0]

    #国内株式のみ抽出

    df = pd.read_excel(url)
    df = df[(df.iloc[:,3] == 'プライム（内国株式）' ) | (df.iloc[:,3] == 'スタンダード（内国株式）' ) | (df.iloc[:,3] == 'グロース（内国株式）' ) ]

    #列名の変更
    df.columns = ['date', 'code', 'office_name', 'market_class', 'industry_detail_code', 'industry_detail', 'industry_code', 'industry','scale_code','scale_class']

    #'date'列をdatetime->strへ型変換
    df['date'] = df['date'].astype(str)
    df['code'] = df['code'].astype(str)
    df['industry_detail_code'] = df['industry_detail_code'].astype(str)
    df['industry_code'] = df['industry_code'].astype(str)
    df['scale_code'] = df['scale_code'].astype(str)

    return df

In [36]:
codes = getStockCodeDataFrame()
codes

,date,code,office_name,market_class,industry_detail_code,industry_detail,industry_code,industry,scale_code,scale_class
0,20240229,1301,極洋,プライム（内国株式）,50,水産・農林業,1,食品,7,TOPIX Small 2
5,20240229,130A,Ｖｅｒｉｔａｓ Ｉｎ Ｓｉｌｉｃｏ,グロース（内国株式）,3250,医薬品,5,医薬品,-,-
19,20240229,1332,ニッスイ,プライム（内国株式）,50,水産・農林業,1,食品,4,TOPIX Mid400
20,20240229,1333,マルハニチロ,プライム（内国株式）,50,水産・農林業,1,食品,4,TOPIX Mid400
31,20240229,135A,ＶＲＡＩＮ Ｓｏｌｕｔｉｏｎ,グロース（内国株式）,5250,情報・通信業,10,情報通信・サービスその他,-,-
...,...,...,...,...,...,...,...,...,...,...
4338,20240229,9993,ヤマザワ,スタンダード（内国株式）,6100,小売業,14,小売,7,TOPIX Small 2
4339,20240229,9994,やまや,スタンダード（内国株式）,6100,小売業,14,小売,7,TOPIX Small 2
4340,20240229,9995,グローセル,プライム（内国株式）,6050,卸売業,13,商社・卸売,7,TOPIX Small 2
4341,20240229,9996,サトー商会,スタンダード（内国株式）,6050,卸売業,13,商社・卸売,-,-


In [37]:
import sqlite3

con = sqlite3.connect("./DB/stockCode.db")
cur  = con.cursor()

create_table_sql ="""
CREATE TABLE stockCode 
(date TEXT,code TEXT, brand TEXT, market TEXT, 
 industry_detail TEXT,industry_detail_code Text,
 industry TEXT,industry_code Text,scale_code Text,scale_class Text
 ,PRIMARY KEY(code))
"""

cur.execute(create_table_sql)
cur.close()
con.close()

In [48]:
#connection確立
con = sqlite3.connect("./DB/stockCode.db")

#一括書込み
df = getStockCodeDataFrame()
df.to_sql('stockCode',con,if_exists='replace',index=None)

#commit
con.commit()
con.close()

In [50]:
con = sqlite3.connect("./DB/stockCode.db")

sql = '''
 SELECT * FROM stockCode 
'''
res = con.execute(sql)
res.fetchone()


('20240229',
 '1301',
 '極洋',
 'プライム（内国株式）',
 '50',
 '水産・農林業',
 '1',
 '食品',
 '7',
 'TOPIX Small 2')

In [51]:
res.close()
con.close()
